In [1]:
import numpy as np
import pprint
import ray

ray.init()

2023-04-06 18:51:48,478	INFO worker.py:1553 -- Started a local Ray instance.


Python version:,3.9.16
Ray version:,2.3.1


In [4]:
from multiagent_env import GridWorldEnv
# Import a Trainable (one of RLlib's built-in algorithms):
# We use the PPO algorithm here b/c its very flexible wrt its supported
# action spaces and model types and b/c it learns well almost any problem.
from ray.rllib.agents.ppo import PPOTrainer, PPOTF1Policy, PPOTF2Policy, PPOTorchPolicy

env = GridWorldEnv()

policies = {
    "policy_1": (PPOTF1Policy, env.observation_space, env.action_space, {}),
}

policy_mapping_fn = lambda agent_id, episode, worker, **kwargs: "policy_1"


In [5]:
config = {
    "env": GridWorldEnv,  # "my_env" <- if we previously have registered the env with `tune.register_env("[name]", lambda config: [returns env object])`.
    "env_config": {},
    "framework": "tf",  # If users have chosen to install torch instead of tf.
    "create_env_on_driver": True,
    "multiagent": {
        "policies": policies,
        "policy_mapping_fn": lambda agent_id, episode, worker, **kwargs: "policy_1",
    },
}
pprint.pprint(config)
# print()
# for agent in env.agents:
#     print(f"{agent} is now mapped to {policy_mapping_fn(agent)}")

rllib_trainer = PPOTrainer(config=config)

{'create_env_on_driver': True,
 'env': <class 'multiagent_env.GridWorldEnv'>,
 'env_config': {},
 'framework': 'tf',
 'multiagent': {'policies': {'policy_1': (<class 'ray.rllib.algorithms.ppo.ppo_tf_policy.PPOTF1Policy'>,
                                          Dict('common': Dict('ore_positions': MultiDiscrete([100 100 100 100 100])), 'friendly': Dict('factory_position': MultiDiscrete([100]), 'robot_ore': MultiDiscrete([51 51 51]), 'robot_positions': MultiDiscrete([101 101 101])), 'opponent': Dict('factory_position': MultiDiscrete([100]), 'robot_ore': MultiDiscrete([51 51 51]), 'robot_positions': MultiDiscrete([101 101 101]))),
                                          MultiDiscrete([10 10 10]),
                                          {})},
                'policy_mapping_fn': <function <lambda> at 0x35464c280>}}
(RolloutWorker pid=13774) Metal device set to: Apple M1 Max
(RolloutWorker pid=13774) 
(RolloutWorker pid=13774) systemMemory: 32.00 GB
(RolloutWorker pid=13774) maxCache

(RolloutWorker pid=13773) 2023-04-06 18:52:30,340	WARNING env.py:296 -- Your MultiAgentEnv <GridWorldEnv instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
2023-04-06 18:52:31,127	WARNING env.py:296 -- Your MultiAgentEnv <GridWorldEnv instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.


Metal device set to: Apple M1 Max

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2023-04-06 18:52:31,934	WARNING util.py:67 -- Install gputil for GPU system monitoring.


In [6]:
for _ in range(5):
    results = rllib_trainer.train()
    print(f"Iteration={rllib_trainer.iteration}: R(\"return\")={results['episode_reward_mean']}")

2023-04-06 18:52:47,156	WARNING deprecation.py:50 -- DeprecationWarning: `_get_slice_indices` has been deprecated. This will raise an error in the future!


Iteration=1: R("return")=-15.114999999999963
Iteration=2: R("return")=-15.264374999999964
Iteration=3: R("return")=-7.760999999999967
Iteration=4: R("return")=-5.039499999999969
Iteration=5: R("return")=-7.223499999999968


In [7]:
obs, _ = env.reset()
env.record()

simulation_time = 100
assert simulation_time > 1

for _ in range(simulation_time-1):
    
    random_sample_action = {
        'team_0': rllib_trainer.compute_single_action(obs["team_0"], policy_id="policy_1"),
        'team_1': rllib_trainer.compute_single_action(obs["team_1"], policy_id="policy_1"),
    }
    obs, reward, _, _, _ = env.step(random_sample_action)
    env.record()
env.render(framerate=2)

Rendering 100 frames...
